# Imports

In [1]:
import warnings # ignore warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import re
import string

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn')

from wordcloud import WordCloud

from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer


from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RepeatedStratifiedKFold, cross_val_score, train_test_split
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, classification_report, plot_confusion_matrix
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Dataframe Initialization and Observations

In [2]:
df = pd.read_csv('./data/judge-1377884607_tweet_product_company.csv',
                 encoding="ISO-8859-1")
df

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
...,...,...,...
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


In [4]:
df.isnull().sum()

tweet_text                                               1
emotion_in_tweet_is_directed_at                       5802
is_there_an_emotion_directed_at_a_brand_or_product       0
dtype: int64

In [5]:
df['emotion_in_tweet_is_directed_at'].value_counts(dropna=False)

NaN                                5802
iPad                                946
Apple                               661
iPad or iPhone App                  470
Google                              430
iPhone                              297
Other Google product or service     293
Android App                          81
Android                              78
Other Apple product or service       35
Name: emotion_in_tweet_is_directed_at, dtype: int64

# Data Cleaning

In [6]:
def process_string(text):
    """This function returns a processed list of words from the given text
    
    This function removes html elements and urls using regular expression, then
    converts string to list of workds, them find the stem of words in the list of words and
    finally removes stopwords and punctuation marks from list of words.
    
    Args:
        text(string): The text from which html elements, urls, stopwords, punctuation are removed and stemmed
        
    Returns:
        clean_text(string): A text formed after text preprocessing.
    """
    
    # Remove any urls from the text
    text = re.sub(r"https:\/\/.*[\r\n]*",
                  "",
                  str(text))
    
    # Remove any urls starting from www. in the text
    text = re.sub(r"www\.\w*\.\w\w\w",
                  "",
                  str(text))
    
    # Remove any html elements from the text
    text = re.sub(r"<[\w]*[\s]*/>",
                  "",
                  str(text))
    
    # Remove prediods  marks
    text = re.sub(r"[\.]*",
                  "",
                  str(text))
    
 
    # Initialize RegexpTokenizer
    pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
    tokenizer = RegexpTokenizer(pattern)

    
    # Tokenize text
    text_tokens = tokenizer.tokenize(text.lower())
    
    lemmatizer  = WordNetLemmatizer()
    # Get english stopwords
    english_stopwords = stopwords.words("english")
    new_list = ["mention", "sxsw", 'link', 'rt', 'quot']
    english_stopwords.extend(new_list)
    
    cleaned_text_tokens = [] # A list to hold cleaned text tokens
    
    for word in text_tokens:
        if((word not in english_stopwords) and # Remove stopwords
            (word not in string.punctuation)): # Remove punctuation marks
                
                lemmas = lemmatizer.lemmatize(word) # Get lemma of the current word
                cleaned_text_tokens.append(lemmas) # Appened lemma word to list of cleaned list
    
    # Combine list into single string
    clean_text = " ".join(cleaned_text_tokens)
    
    return clean_text

In [7]:
#
df['tweet_text'] = df['tweet_text'].apply(process_string)

In [8]:
# Categorize Tweets by 'Manufacturer' based on if Tweet contains certain words
is_apple = ['ipad', 'iphone', 'apple', 'mac', 'ios']
is_google = ['google', 'pixel', 'android']

def apple_sorter(x):
    for i in is_apple:
        if i.lower() in x.lower():
            return 'Apple'
        else:
            continue
        
def google_sorter(x):
    for i in is_google:
        if i.lower() in x.lower():
            return 'Google'
        else:
            continue

In [9]:
df['Manufacturer'] = df['tweet_text'].apply(apple_sorter)
df['Google'] = df['tweet_text'].apply(google_sorter)
df['Manufacturer'] = df['Manufacturer'].combine_first(df['Google'])
df.drop('Google', axis=1, inplace=True)
df.drop('emotion_in_tweet_is_directed_at', axis=1, inplace=True)

In [10]:
df.head()

,tweet_text,is_there_an_emotion_directed_at_a_brand_or_product,Manufacturer
0,wesley g iphone hr tweeting rise austin dead n...,Negative emotion,Apple
1,jessedee know fludapp awesome ipad iphone app ...,Positive emotion,Apple
2,swonderlin wait ipad also sale,Positive emotion,Apple
3,hope year's festival crashy year's iphone app,Negative emotion,Apple
4,sxtxstate great stuff fri marissa mayer google...,Positive emotion,Google


In [11]:
df['Manufacturer'].value_counts(dropna=False)

Apple     5548
Google    2760
NaN        785
Name: Manufacturer, dtype: int64

In [12]:
df['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [ ]:
df = df[df['is_there_an_emotion_directed_at_a_brand_or_product'] != "I can't tell"]

In [ ]:
df['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

In [ ]:
def process_string(text):
    """This function returns a processed list of words from the given text
    
    This function removes html elements and urls using regular expression, then
    converts string to list of workds, them find the stem of words in the list of words and
    finally removes stopwords and punctuation marks from list of words.
    
    Args:
        text(string): The text from which html elements, urls, stopwords, punctuation are removed and stemmed
        
    Returns:
        clean_text(string): A text formed after text preprocessing.
    """
    
    # Remove any urls from the text
    text = re.sub(r"https:\/\/.*[\r\n]*",
                  "",
                  str(text))
    
    # Remove any urls starting from www. in the text
    text = re.sub(r"www\.\w*\.\w\w\w",
                  "",
                  str(text))
    
    # Remove any html elements from the text
    text = re.sub(r"<[\w]*[\s]*/>",
                  "",
                  str(text))
    
    # Remove prediods  marks
    text = re.sub(r"[\.]*",
                  "",
                  str(text))
    
 
    # Initialize RegexpTokenizer
    pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
    tokenizer = RegexpTokenizer(pattern)

    
    # Tokenize text
    text_tokens = tokenizer.tokenize(text.lower())
    
    lemmatizer  = WordNetLemmatizer()
    # Get english stopwords
    english_stopwords = stopwords.words("english")
    new_list = ["mention", "sxsw"]
    english_stopwords.extend(new_list)
    
    cleaned_text_tokens = [] # A list to hold cleaned text tokens
    
    for word in text_tokens:
        if((word not in english_stopwords) and # Remove stopwords
            (word not in string.punctuation)): # Remove punctuation marks
                
                lemmas = lemmatizer.lemmatize(word) # Get lemma of the current word
                cleaned_text_tokens.append(lemmas) # Appened lemma word to list of cleaned list
    
    # Combine list into single string
    clean_text = " ".join(cleaned_text_tokens)
    
    return clean_text

In [ ]:
X_train = X_train['tweet_text'].apply(process_string)
X_test = X_test['tweet_text'].apply(process_string)

In [ ]:
label_encoder = preprocessing.LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [ ]:
# df['is_there_an_emotion_directed_at_a_brand_or_product'] = label_encoder.fit_transform(df['is_there_an_emotion_directed_at_a_brand_or_product'])
# df['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()


In [ ]:
tf_idf = TfidfVectorizer()

X_train = tf_idf.fit_transform(X_train.tolist())
X_test = tf_idf.transform(X_test.tolist())

print(X_train.shape, X_test.shape)

In [ ]:
# Create Dummy/Baseliner
estimator = DummyClassifier(strategy='most_frequent')
estimator.fit(X_train, y_train)

# Get predictions with dummy model
y_pred = estimator.predict(X_test)


# Set figsize and font scale
sns.set(rc={'figure.figsize':(9, 4)})
sns.set(font_scale=1)

# Set diply labesl for confusion matrix
display_labels = ['-', 'Neutral', '+']

# Plot a confusion matrix on the test data
plot_confusion_matrix(estimator=estimator,
                      X=X_test,
                      y_true=y_test,
                      display_labels=display_labels)

# Save confusion matrix as png and place it in the images folder
plt.savefig('images/Confusion_Matrix_Dummy');

target_names = ['-', 'Neutral', '+']
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
pipe = Pipeline(steps=[
    ('estimator', LogisticRegression(random_state=42))
])



param_grid = {}
param_grid['estimator__C'] = [100, 10, 1.0, 0.1, 0.01]
param_grid['estimator__solver'] = ['newton-cg', 'lbfgs', 'liblinear']
param_grid['estimator__penalty'] = ['l2']



grid_search = GridSearchCV(estimator=pipe, 
                           param_grid=param_grid, 
                           cv=RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42), 
                           return_train_score=True, 
                           scoring='accuracy', 
                           n_jobs=-1,
                           verbose=2)



# Fit models run gridsearch
grid_search.fit(X_train, y_train)

# Mean training score
grid_train_score = np.mean(grid_search.cv_results_['mean_train_score'])

# Mean test score
grid_test_score = grid_search.score(X_test, y_test)

best_grid = grid_search.best_estimator_
best_grid.fit(X_train, y_train)
y_pred = best_grid.predict(X_test)



print(f"Mean Training Score: {grid_train_score:.2%}\n")
print(f"Mean Test Score: {grid_test_score:.2%}\n")

print(f"Optimal Parameters: {grid_search.best_params_}\n")
print(f"Testing Accuracy: {grid_search.best_score_:.2%}\n")


# Set figsize and font scale
sns.set(rc={'figure.figsize':(9, 4)})
sns.set(font_scale=1)

# Set diply labesl for confusion matrix
display_labels = ['-', 'Neutral', '+']

# Plot a confusion matrix on the test data
plot_confusion_matrix(estimator=estimator,
                      X=X_test,
                      y_true=y_test,
                      display_labels=display_labels)


target_names = ['-', 'Neutral', '+']
print(classification_report(y_test, y_pred, target_names=target_names))

# Save confusion matrix as png and place it in the images folder
plt.savefig('images/Confusion_Matrix_LogReg');

In [ ]:
def text_contains(df):
    i = 0
    if df["tweet_text"][i].contains(['ipad', 'iphone', 'apple', 'mac', 'ios']) in df:
        df['emotion_in_tweet_is_directed_at'][i] = 'Apple'
        i += 1
    elif df["tweet_text"][i].contains(['google', 'andriod', 'pixel']):
        df['emotion_in_tweet_is_directed_at'][i] = 'Google'
        i += 1
    else:
        i += 1

In [ ]:
def text_contains(df):
    i = 0
    for df["tweet_text"][i] in df:
        if 'ipad' or 'iphone' or 'apple' or 'mac' or 'ios' in df["tweet_text"][i]:
            df['emotion_in_tweet_is_directed_at'][i] = 'Apple'
            i += 1
        elif 'google' or 'andriod' or 'pixel' in df["tweet_text"][i]:
            df['emotion_in_tweet_is_directed_at'][i] = 'Google'
            i += 1
        else:
            i += 1

In [ ]:
# df.drop(df.index[-1])

In [ ]:
X = df.drop(columns=["emotion_in_tweet_is_directed_at", "is_there_an_emotion_directed_at_a_brand_or_product"])
y = df.drop(columns=["emotion_in_tweet_is_directed_at", "tweet_text"])

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.33, 
                                                    random_state=42)

In [ ]:
label_encoder = preprocessing.LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

print(y_train), print(y_test)

In [ ]:
tf_idf = TfidfVectorizer()

X_train = tf_idf.fit_transform(X_train["tweet_text"].tolist())
X_test = tf_idf.transform(X_test["tweet_text"].tolist())

print(X_train.shape, X_test.shape)

In [ ]:
# Create Dummy/Baseliner
estimator = DummyClassifier(strategy='most_frequent')
estimator.fit(X_train, y_train)

# Get predictions with dummy model
y_pred = estimator.predict(X_test)


# Set figsize and font scale
sns.set(rc={'figure.figsize':(9, 4)})
sns.set(font_scale=1)

# Set diply labesl for confusion matrix
display_labels = ['Negative emotion', 'No emotion toward brand or product', 'Positive emotion']

# Plot a confusion matrix on the test data
plot_confusion_matrix(estimator=estimator,
                      X=X_test,
                      y_true=y_test,
                      display_labels=display_labels)

# Save confusion matrix as png and place it in the images folder
plt.savefig('images/Confusion_Matrix_Dummy');

target_names = ['Negative emotion', 'No emotion toward brand or product', 'Positive emotion']
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
df['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

In [ ]:
pipe = Pipeline(steps=[
    ('estimator', LogisticRegression(random_state=42))
])



param_grid = {}
param_grid['estimator__C'] = [100, 10, 1.0, 0.1, 0.01]
param_grid['estimator__solver'] = ['newton-cg', 'lbfgs', 'liblinear']
param_grid['estimator__penalty'] = ['l2']



grid_search = GridSearchCV(estimator=pipe, 
                           param_grid=param_grid, 
                           cv=RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42), 
                           return_train_score=True, 
                           scoring='accuracy', 
                           n_jobs=-1,
                           verbose=2)



# Fit models run gridsearch
grid_search.fit(X_train, y_train)

# Mean training score
grid_train_score = np.mean(grid_search.cv_results_['mean_train_score'])

# Mean test score
grid_test_score = grid_search.score(X_test, y_test)

best_grid = grid_search.best_estimator_
best_grid.fit(X_train, y_train)
y_pred = best_grid.predict(X_test)



print(f"Mean Training Score: {grid_train_score:.2%}\n")
print(f"Mean Test Score: {grid_test_score:.2%}\n")

print(f"Optimal Parameters: {grid_search.best_params_}\n")
print(f"Testing Accuracy: {grid_search.best_score_:.2%}\n")


# Set figsize and font scale
sns.set(rc={'figure.figsize':(9, 4)})
sns.set(font_scale=1)

# Set diply labesl for confusion matrix
display_labels = ['-', 'Neutral', '+']

# Plot a confusion matrix on the test data
plot_confusion_matrix(estimator=estimator,
                      X=X_test,
                      y_true=y_test,
                      display_labels=display_labels)


target_names = ['-', 'Neutral', '+']
print(classification_report(y_test, y_pred, target_names=target_names))

# Save confusion matrix as png and place it in the images folder
plt.savefig('images/Confusion_Matrix_LogReg');

In [ ]:
# # Create an intance of the RegexpTokenizer with the variable name `tokenizer`
# # The regex pattern should select all words with three or more characters
# # pattern = r"(?u)\w{3,}"
# pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
# tokenizer = RegexpTokenizer(pattern)

# # Create a list of stopwords in English
# stopwords_list = stopwords.words('english')

# # Create an instance of nltk's PorterStemmer with the variable name `stemmer`
# lemmatizer = WordNetLemmatizer()

In [ ]:
# def get_wordnet_pos(treebank_tag):
#     '''
#     Translate nltk POS to wordnet tags
#     '''
#     if treebank_tag.startswith('J'):
#         return wordnet.ADJ
#     elif treebank_tag.startswith('V'):
#         return wordnet.VERB
#     elif treebank_tag.startswith('N'):
#         return wordnet.NOUN
#     elif treebank_tag.startswith('R'):
#         return wordnet.ADV
#     else:
#         return wordnet.NOUN

In [ ]:
# def preprocess_text(df, tokenizer, stopwords_list, lemmatizer):
#     # Standardize case (lowercase the text)
#     lowered = df[col].lower()
    
#     # Tokenize text using `tokenizer`
#     tokens = tokenizer.tokenize(lowered)
    
#     # Remove stopwords using `stopwords_list`
#     stopped_tokens = [word for word in tokens if word not in stopwords_list]
    
#     pos = pos_tag(stopped_tokens)
#     pos = [(word[0], get_wordnet_pos(word[1])) for word in pos]
    
#     # Stem the tokenized text using `stemmer`
#     lemmas = [lemmatizer.lemmatize(word[0], word[1]) for word in pos]
    
#     # Return the preprocessed text
#     return lemmas

In [ ]:
def process_string(text):
    """This function returns a processed list of words from the given text
    
    This function removes html elements and urls using regular expression, then
    converts string to list of workds, them find the stem of words in the list of words and
    finally removes stopwords and punctuation marks from list of words.
    
    Args:
        text(string): The text from which hrml elements, urls, stopwords, punctuation are removed and stemmed
        
    Returns:
        clean_text(string): A text formed after text preprocessing.
    """
    
    # Remove any urls from the text
    text = re.sub(r"https:\/\/.*[\r\n]*",
                  "",
                  str(text))
    
    # Remove any urls starting from www. in the text
    text = re.sub(r"www\.\w*\.\w\w\w",
                  "",
                  str(text))
    
    # Remove any html elements from the text
    text = re.sub(r"<[\w]*[\s]*/>",
                  "",
                  str(text))
    
    # Remove prediods  marks
    text = re.sub(r"[\.]*",
                  "",
                  str(text))
    
 
    # Initialize RegexpTokenizer
    pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
    tokenizer = RegexpTokenizer(pattern)

    
    # Tokenize text
    text_tokens = tokenizer.tokenize(text.lower())
    
    lemmatizer  = WordNetLemmatizer()
    # Get english stopwords
    english_stopwords = stopwords.words("english")
    
    cleaned_text_tokens = [] # A list to hold cleaned text tokens
    
    for word in text_tokens:
        if((word not in english_stopwords) and # Remove stopwords
            (word not in string.punctuation)): # Remove punctuation marks
                
                lemmas = lemmatizer.lemmatize(word) # Get lemma of the current word
                cleaned_text_tokens.append(lemmas) # Appened lemma word to list of cleaned list
    
    # Combine list into single string
    clean_text = " ".join(cleaned_text_tokens)
    
    return clean_text

df['tweet_text'] = df['tweet_text'].apply(process_string)
df

In [ ]:
df['tweet_text'] = df['tweet_text'].apply(process_string)
df

In [ ]:
df['tweet_text'][9092]

In [ ]:
df = df.reset_index()

In [ ]:
df

In [ ]:
df['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts(dropna=False, normalize=True)

In [ ]:
#create a new data frame with "id" and "comment" fields
df_subset = df[['index', 'is_there_an_emotion_directed_at_a_brand_or_product']].copy()
#data clean-up
#remove all non-aphabet characters
df_subset['is_there_an_emotion_directed_at_a_brand_or_product'] = df_subset['is_there_an_emotion_directed_at_a_brand_or_product'].str.replace("[^a-zA-Z#]", " ")
#covert to lower-case
df_subset['is_there_an_emotion_directed_at_a_brand_or_product'] = df_subset['is_there_an_emotion_directed_at_a_brand_or_product'].str.casefold()
print (df_subset.head(10))

In [ ]:
# set up empty dataframe for staging output
df1 = pd.DataFrame()
df1['index'] = ['99999999999']
df1['sentiment_type']='NA999NA'
df1['sentiment_score']=0

In [ ]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

print('Processing sentiment analysis...')
sid = SentimentIntensityAnalyzer()
t_df = df1
for index, row in df_subset.iterrows():
    scores = sid.polarity_scores(row[1])
    for key, value in scores.items():
        temp = [key, value, row[0]]
        df1['index'] = row[0]
        df1['sentiment_type'] = key
        df1['sentiment_score'] = value
        t_df = t_df.append(df1)
#remove dummy row with row_id = 99999999999
t_df_cleaned = t_df[t_df.index != '99999999999']
#remove duplicates if any exist
t_df_cleaned = t_df_cleaned.drop_duplicates()
# only keep rows where sentiment_type = compound
t_df_cleaned = t_df[t_df.sentiment_type == 'compound']
print(t_df_cleaned.head(10))

In [ ]:
#merge dataframes
df = pd.merge(df, t_df_cleaned, on='index', how='inner')
print(df_output.head(10))

In [ ]:
df.head(50)

In [ ]:
process_string(df['tweet_text'][4])

In [ ]:
label_encoder = preprocessing.LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

print(y_train), print(y_test)

In [ ]:
from PIL import Image
from wordcloud import WordCloud #library that allows word cloud to be generated

def generate_word_cloud(dataframe_df,column_name,img_file=None,background_color="black",max_words=2000):
    """ This function generates word cloud
    
    Args:
        dataframe: the datafram object which contains column whose word cloud is to be generated
        column_name: the name of the column in dataframw whose cloud is to be generated
        
    Returns:
        None
    """
    #convert the column specified by column_name in dataset_df(dataframe) to list
    sentences_list = dataframe_df[column_name].tolist()
    
    #convert a list to sentence
    #sentences_single_string = " ".join(sentences_list)
    sentences_single_string = " ".join(map(str,sentences_list))
    
    #plot word cloud
    plt.figure(figsize=(15,15))
    
    img_mask = None
    
    if img_file != None:
        img_mask = np.array(Image.open(img_file))

    wc = WordCloud(mask=img_mask,
                   max_words=max_words,
                   background_color=background_color)        
        
    plt.imshow(wc.generate(sentences_single_string),interpolation="bilinear")
    plt.title("Word Cloud For " +  column_name)
    plt.show()

In [ ]:
import time
tic= time.time()
generate_word_cloud(X_train,
                    "tweet_text",
                    background_color="black")
toc = time.time()
diff = 1000*(toc - tic)
print("Total Time Taken: " + str(diff) + " ms")

In [ ]:
X_train.to_csv("./data/X_train.csv")
X_test.to_csv("./data/X_test.csv")

In [ ]:
tf_idf = TfidfVectorizer()


X_train = tf_idf.fit_transform(X_train["tweet_text"].tolist())
X_test = tf_idf.transform(X_test["tweet_text"].tolist())

print(X_train.shape, X_test.shape)

In [ ]:
for record in df:
    if 'ipad' in record['tweet_text']:
        record['Manufacturer'] == 'Apple')
    elif 'google' in record['tweet_text']:
        record['Manufacturer'].replace('NaN', 'Google')

In [ ]:
apple = ['ipad', 'iphone', 'apple', 'mac', 'ios']
google = ['google', 'andriod', 'pixel', 'windows']

In [ ]:
df.isnull().sum()